In [1]:
pwd

'/lustre06/project/6001735/detests_2022/notebooks/juan'

In [2]:
import pandas as pd
import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, accuracy_score, precision_recall_curve, f1_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from nltk.corpus import stopwords
# import matplotlib.pyplot as plt

In [3]:
import sys
import importlib
sys.path.append("../../library")
import utils, juan
from juan import *

In [4]:
data = pd.read_csv("../../data/task_2.csv")
show(data)

(3817, 11)


,sentence,xenophobia,suffering,economic,migration,culture,benefits,health,security,dehumanisation,others
0,La solución es desarrollar el pensamiento crít...,0,0,0,0,0,0,0,0,0,0
1,Hay que enseñar que la magia no existe.,0,0,0,0,0,0,0,0,0,0
2,Que todos los avances de la humanidad siempre ...,0,0,0,0,0,0,0,0,0,0
3,Enseñar en las escuelas la historia de las rel...,0,0,0,0,0,0,0,0,0,0
4,Desde las religiones de la edad de piedra hast...,0,0,0,0,0,0,0,0,0,0


In [5]:
labels = data.iloc[:, 1:]
show(labels)

(3817, 10)


,xenophobia,suffering,economic,migration,culture,benefits,health,security,dehumanisation,others
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0


In [6]:
%%time
tokenizer = transformers.AutoTokenizer.from_pretrained("../../assets/beto/tokenizer")

CPU times: user 102 ms, sys: 76.6 ms, total: 178 ms
Wall time: 679 ms


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
model = transformers.AutoModel.from_pretrained("../../assets/beto/model")
model.to(device)

X = data.sentence.sample(100)
Y = labels.sample(100)

importlib.reload(juan)
importlib.reload(utils)
clf = juan.BetoMTL(tokenizer, model)
clf.to(device)

# clf.entrenar(
#     X,
#     Y,
#     epochs = 5,
# #     freeze_encoder = True,
# #     weight_decay = 1000,
# #     class_weights = "balanced"
# )


# predictions = clf.predecir_proba(X)
# # predictions = clf.predecir(X)
# predictions[:5]

results = utils.validate_MTL_juan(
    X,
    Y, 
#     X, Y,
    clf, 
#     epochs = 5,
#     freeze_encoder = True,
    progress_bar = False
#     weight_decay = 100,
#     class_weights = "balanced"
)
results

Some weights of the model checkpoint at ../../assets/beto/model were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


X (100,)
Y (100, 10)
Cross-validation process started...
*** fold 1 / 5
    training model...
Training model...
    first step, train loss:0.5519
    epoch: 1, train loss:0.2781, train accuracy: 0.9675
    epoch: 2, train loss:0.1141, train accuracy: 0.9812
    generating predictions on the train set...
    generating predictions on the test set...
    Total runtime: 0.05 minutes
*** fold 2 / 5
    training model...
Training model...
    first step, train loss:0.5694
    epoch: 1, train loss:0.2858, train accuracy: 0.9563
    epoch: 2, train loss:0.1240, train accuracy: 0.9788
    generating predictions on the train set...
    generating predictions on the test set...
    Total runtime: 0.10 minutes
*** fold 3 / 5
    training model...
Training model...
    first step, train loss:0.6369
    epoch: 1, train loss:0.2962, train accuracy: 0.9550
    epoch: 2, train loss:0.1213, train accuracy: 0.9788
    generating predictions on the train set...
    generating predictions on the test set.

,test_accuracy,test_f1,test_precision,test_recall,train_accuracy,train_f1,train_precision,train_recall
column,,,,,,,,
benefits,0.96,0.0,0.0,0.0,0.96,0.0,0.0,0.0
culture,0.98,0.0,0.0,0.0,0.98,0.0,0.0,0.0
dehumanisation,0.99,0.0,0.0,0.0,0.99,0.0,0.0,0.0
economic,1.00,0.0,0.0,0.0,1.00,0.0,0.0,0.0
health,1.00,0.0,0.0,0.0,1.00,0.0,0.0,0.0
migration,0.93,0.0,0.0,0.0,0.93,0.0,0.0,0.0
others,0.99,0.0,0.0,0.0,0.99,0.0,0.0,0.0
security,0.95,0.0,0.0,0.0,0.95,0.0,0.0,0.0
suffering,1.00,0.0,0.0,0.0,1.00,0.0,0.0,0.0


In [15]:
np.logical_or.reduce(Y, axis = 1).mean()

0.16

In [18]:
1 - Y.sum(axis = 0).sort_index()/100

benefits          0.96
culture           0.98
dehumanisation    0.99
economic          1.00
health            1.00
migration         0.93
others            0.99
security          0.95
suffering         1.00
xenophobia        1.00
dtype: float64